# MAC 5768 - Projeto EP - Parte 1 




#### Grupo:
- Ysabelle Pinheiro de Sousa - ysabellesousa@gmail.com - 11341314
- Roberto Gaui 
- Joaquin Lorenzo Pereira Condori


####  Link para repositório projeto: 

https://github.com/YsabelleSousa/mac_5768_ep_visao_proc_imagens

#### Instrução para download dos dados:

Link: https://drive.google.com/file/d/1j9RSKIKmOTkDJt0_ASO3968sTrslPZs1/view?usp=sharing

**1)** Gerar o link compartilhável da pasta zipada no google drive

**2)** Atribuir o ID que está no link compartilhável à variável "id_google_drive" na seção "Baixando Dados"

**3)** Escolher o path de destino e atribuir à variável "path_destination" na seção "Baixando Dados"

**4)** Executar a função "download_file_from_google_drive"


## Importando Bibliotecas

In [5]:
import pandas as pd
from utils import utils
from google_drive_downloader import GoogleDriveDownloader as gdd

import os
import sys

## Baixando Dados

In [6]:
id_google_drive = '1XkKYNmaP6wIfuiwcDloAgaQc9POxGosk'
path_destination = 'data/EP1.zip'


In [ ]:
gdd.download_file_from_google_drive(file_id = id_google_drive,
                                    dest_path = path_destination,
                                    unzip = True,
                                    showsize = True,
                                    overwrite=False)


## Extraindo informações para DF

In [7]:
df_dados = pd.DataFrame(columns = ['classe','num_objeto','tp_fundo','dia_noite','dentro_fora','num_repeticao','extensao'])

In [20]:
for root, subdirs, files in os.walk(os.path.abspath('data/EP1_V2')):
    
    for file in files:
        if file.count('_') == 5:
            df_dados = df_dados.append({
            
                        'classe': file.split('_')[0],
                        'num_objeto': file.split('_')[1],
                        'tp_fundo': file.split('_')[2],
                        'dia_noite': file.split('_')[3],
                        'dentro_fora': file.split('_')[4],
                        'num_repeticao': file.split('_')[5].split('.')[0],
                        'extensao': file.split('_')[5].split('.')[1],
            }, ignore_index=True)

In [21]:
df_dados

,classe,num_objeto,tp_fundo,dia_noite,dentro_fora,num_repeticao,extensao
0,colher,3,laranja,noite,fora,3,jpg
1,colher,2,laranja,dia,dentro,2,jpg
2,colher,1,azul,dia,dentro,1,jpg
3,colher,1,azul,noite,dentro,1,jpg
4,colher,1,laranja,dia,fora,3,jpg
...,...,...,...,...,...,...,...
319,celular,2,branco,dia,fora,3,jpg
320,celular,3,branco,noite,dentro,2,jpg
321,celular,3,laranja,dia,fora,1,jpg
322,celular,2,branco,dia,fora,2,jpg


## Tabela Sumária Global

In [22]:
df_tbl_sumaria = pd.DataFrame(columns=['num_classes','num_imagens','tamanho_base_mb','extensoes_imagens'])
                              

In [26]:
df_tbl_sumaria = df_tbl_sumaria.append({
    'num_classes': df_dados['classe'].nunique(),
    'num_imagens': len(df_dados),
    'tamanho_base_mb': utils.return_folder_size_mb('data/EP1_V2'),
    'extensoes_imagens': df_dados['extensao'].unique()
}, ignore_index=True)

In [27]:
df_tbl_sumaria

,num_classes,num_imagens,tamanho_base_mb,extensoes_imagens
0,3,324,665.77,[jpg]


## Tabela Sumária Detalhada

In [28]:
df_tbl_sumaria_detalhada = df_dados.groupby('classe', as_index=False).agg({
    'num_objeto': pd.Series.nunique,
    'tp_fundo': lambda x: dict(x.value_counts()),
    'dia_noite': lambda x: dict(x.value_counts()),
    'dentro_fora': lambda x: dict(x.value_counts()),
    'num_repeticao': 'nunique',
    'extensao': 'count'
})

In [29]:
df_tbl_sumaria_detalhada

,classe,num_objeto,tp_fundo,dia_noite,dentro_fora,num_repeticao,extensao
0,caneca,3,"{'azul': 36, 'laranja': 36, 'branco': 36}","{'noite': 54, 'dia': 54}","{'fora': 54, 'dentro': 54}",3,108
1,celular,3,"{'azul': 36, 'branco': 36, 'laranja': 36}","{'noite': 54, 'dia': 54}","{'fora': 54, 'dentro': 54}",3,108
2,colher,3,"{'azul': 36, 'laranja': 36, 'branco': 36}","{'noite': 54, 'dia': 54}","{'fora': 54, 'dentro': 54}",3,108


In [30]:
df_tbl_sumaria_detalhada.columns = ['classe','qtd_objetos','variacoes_fundo','variacoes_iluminacao_externa','variacoes_iluminacao_local','qtd_repeticoes','total_amostras']

In [31]:
df_tbl_sumaria_detalhada

,classe,qtd_objetos,variacoes_fundo,variacoes_iluminacao_externa,variacoes_iluminacao_local,qtd_repeticoes,total_amostras
0,caneca,3,"{'azul': 36, 'laranja': 36, 'branco': 36}","{'noite': 54, 'dia': 54}","{'fora': 54, 'dentro': 54}",3,108
1,celular,3,"{'azul': 36, 'branco': 36, 'laranja': 36}","{'noite': 54, 'dia': 54}","{'fora': 54, 'dentro': 54}",3,108
2,colher,3,"{'azul': 36, 'laranja': 36, 'branco': 36}","{'noite': 54, 'dia': 54}","{'fora': 54, 'dentro': 54}",3,108
